# h

As in the previous workbooks, I'll start by importing the names of resolved galaxies we've identified. 

In [1]:
import os 

datapath = 'Sim Data (Clusters; 800, Seed; 2639)' # all of the data is within a folder in this .ipynb file's directory
GalaxyNames = []
for clusterFile in os.listdir(datapath + '/Star Clusters'):
    GalaxyNames.append(clusterFile[:-4]) # this gets the name of the cluster, without the `.txt' file extension

To begin with, I'll look at the period-luminosity relationship as a means of distance estimation. That is, for all of the variable stars in a galaxy, I'll estimate their intrinsic luminosity and compare that against their observed flux (via the inverse square law) to calculate distance. 

In [2]:
import pandas as pd
import numpy as np
from astropy.timeseries import LombScargle

for name in [GalaxyNames[0]]:
    galaxdata = pd.read_csv(datapath + f'/Star Clusters/{name}.txt', delimiter=' ')
    variables = galaxdata['Variable?']
    variableindexes = [i for i, x in enumerate(variables) if x == 1]
    variablenames = galaxdata['Name'][variableindexes].to_numpy()
    periods = np.zeros(len(variablenames))
    for star in variablenames:
        photometryData = pd.read_csv(datapath + f"/Variable Star Data/{star}.txt")
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'Sim Data (Clusters; 800, Seed; 2639)/Variable Star Data/S312703.txt'